### Empirical Asset Pricing Part 1 - Panelized Linear Models
Author: Ren Yang

In [ ]:
import pandas as pd
import scipy.stats as ss
import numpy as np
import datetime
import xgboost as xgb


from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [ ]:
# FOR COLAB
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/Students/'
file_name='openap_macro_merged.parquet.gzip'
stock_data=pd.read_parquet(data_dir+file_name)

In [ ]:
stock_sub=stock_data[stock_data['DateYM']>np.datetime64('2000-01-01')].reset_index()

In [ ]:
stock_sub_3=stock_sub.set_index(['permno','DateYM'],verify_integrity=True).drop('index', axis=1)
for i in stock_sub_3.columns[:-1]:
  if stock_sub_3[i].nunique()==2: # Check if column is binary
    stock_sub_3[i].fillna(0,inplace=True)
    stock_sub_3[i]=np.where(stock_sub_3[i]==0,-1,1) 
  else:
    stock_sub_3[i]=stock_sub_3[i].groupby('DateYM').apply(lambda x: x.fillna(0) if  np.isnan(x.median()) else x.fillna(x.median()))# fill non binary column NaNs with period cross-sectional median. if period cross-sectional median donesn't exist, fill with 0                    
    stock_sub_3[i]=stock_sub_3[i].groupby('DateYM').rank(pct=True).transform(lambda x:2*((x-x.min())/(x.max()-x.min()))-1) 
                      

In [ ]:
stock_sub_3

AbnormalAccruals  Accruals  AccrualsBM  Activism1  \
permno DateYM                                                          
10001  2000-01-31          0.735452  0.803083          -1  -0.959920   
       2000-02-29          0.738235  0.805293          -1  -0.959997   
       2000-03-31          0.743596  0.806560          -1  -0.959511   
       2000-04-30          0.745952  0.812163          -1  -0.959353   
       2000-05-31          0.748863  0.813263          -1  -0.958202   
...                             ...       ...         ...        ...   
93436  2020-08-31          0.825222 -0.389954          -1  -0.822600   
       2020-09-30          0.825771 -0.394193          -1  -0.822602   
       2020-10-31          0.828104 -0.400736          -1  -0.822605   
       2020-11-30          0.830327 -0.411127          -1  -0.822608   
       2020-12-31          0.832559 -0.419940          -1  -0.822611   

                   Activism2     AdExp    AgeIPO        AM  AnalystRevision  \
permno DateYM                                                                 
10001  2000-01-31  -0.993079 -0.787514 -0.985663  0.378678         0.434580   
       2000-02-29  -0.992910 -0.786478 -0.990578  0.413982         0.399833   
       2000-03-31  -0.992899 -0.783664 -0.994734  0.419579         0.312066   
       2000-04-30  -0.991549 -0.784762 -0.993284  0.439177         0.417057   
       2000-05-31  -0.990785 -0.783664 -0.989440  0.423080         0.414053   
...                      ...       ...       ...       ...              ...   
93436  2020-08-31  -0.962013  0.171236 -0.902696 -0.458939         0.000060   
       2020-09-30  -0.962015  0.176616 -0.897891 -0.494035         0.000059   
       2020-10-31  -0.962017  0.184759 -0.895071 -0.465578         0.000058   
       2020-11-30  -0.962021  0.198948 -0.890595 -0.434602         0.000056   
       2020-12-31  -0.962024  0.206197 -0.884879 -0.437676         0.000055   

                   AnalystValue  ...      sic2  macro_tbl  macro_svar  \
permno DateYM                    ...                                    
10001  2000-01-31      0.618353  ... -0.076812  -1.000000   -1.000000   
       2000-02-29      0.620405  ... -0.073927  -0.991678   -0.991678   
       2000-03-31      0.624611  ... -0.075357  -0.993068   -0.993068   
       2000-04-30      0.629123  ... -0.072577  -0.985181   -0.985181   
       2000-05-31      0.634666  ... -0.070512  -0.993068   -0.993068   
...                         ...  ...       ...        ...         ...   
93436  2020-08-31      0.392952  ... -0.063564   0.949417    0.949417   
       2020-09-30      0.397174  ... -0.053693   0.919277    0.919277   
       2020-10-31      0.403338  ... -0.041696   0.877195    0.877195   
       2020-11-30      0.411591  ... -0.025178   0.814710    0.814710   
       2020-12-31      0.418309  ... -0.011655   0.763343    0.763343   

                   macro_ntis  macro_bm  macro_dp  macro_ep  macro_tms  \
permno DateYM                                                            
10001  2000-01-31   -1.000000 -1.000000 -1.000000 -1.000000  -1.000000   
       2000-02-29   -0.991678 -0.991678 -0.991678 -0.991678  -0.991678   
       2000-03-31   -0.993068 -0.993068 -0.993068 -0.993068  -0.993068   
       2000-04-30   -0.985181 -0.985181 -0.985181 -0.985181  -0.985181   
       2000-05-31   -0.993068 -0.993068 -0.993068 -0.993068  -0.993068   
...                       ...       ...       ...       ...        ...   
93436  2020-08-31    0.949417  0.949417  0.949417  0.949417   0.949417   
       2020-09-30    0.919277  0.919277  0.919277  0.919277   0.919277   
       2020-10-31    0.877195  0.877195  0.877195  0.877195   0.877195   
       2020-11-30    0.814710  0.814710  0.814710  0.814710   0.814710   
       2020-12-31    0.763343  0.763343  0.763343  0.763343   0.763343   

                   macro_dfy    retadj  
permno DateYM                           
10001  2000-01-31  -1.000000 -0.044118  
       2000-02-29 

In [ ]:
df_rank=stock_sub_3[stock_sub_3.index.get_level_values(1)==pd.to_datetime('2020-12-31')] # rank based on the latest day
df_rank['mvel_rank']=df_rank['mvel1'].rank()
top_1000_permno=df_rank[df_rank['mvel_rank']>4746].index.get_level_values(0)
bot_1000_permno=df_rank[df_rank['mvel_rank']<=1000].index.get_level_values(0)


# slice top 1000 stocks
top_1000_df=stock_sub_3.loc[top_1000_permno, :]

# slice bot 1000 stocks
bot_1000_df=stock_sub_3.loc[bot_1000_permno,:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# train for top 1000 stocks using random forest 


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_top_1000={}

cycle_prediction_results_top_1000={}

while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=top_1000_df.loc[top_1000_df.index.get_level_values(1)<end_of_validation]
  
  cycle_test=top_1000_df.loc[(top_1000_df.index.get_level_values(1)>=start_of_test) & (top_1000_df.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 

  cv = model_selection.PredefinedSplit(cycle_train_val['test_fold'])
  
  enet=linear_model.ElasticNetCV(cv=cv,random_state=344, max_iter=10000)
  enet.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  print(f'Cycle{cycle_counter} model trained')

  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  


  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
    
     count+=1
     
     y_pred=enet.predict(X=df.drop(columns=['retadj']))
     
     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)
     
     cycle_prediction_results_top_1000[df.index.get_level_values(1)[0]]=y_pred
  
  cycle_r_2_results_top_1000[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)








Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


In [ ]:
cycle_r_2_results_top_1000

{'Cycle0Average Monthly R2:': -0.35773038367758686,
 'Cycle1Average Monthly R2:': -0.5695887238165801,
 'Cycle2Average Monthly R2:': -0.42707072699381093,
 'Cycle3Average Monthly R2:': -0.16760126647811546,
 'Cycle4Average Monthly R2:': -0.16967593735250477,
 'Cycle5Average Monthly R2:': -0.18029309675563718,
 'Cycle6Average Monthly R2:': -0.24758288031684383,
 'Cycle7Average Monthly R2:': -0.17951036558345393,
 'Cycle8Average Monthly R2:': -0.028061319848741207,
 'Cycle9Average Monthly R2:': -0.3472202400217456,
 'Cycle10Average Monthly R2:': -0.24136415708072853}

In [ ]:
cycle_prediction_results_top_1000

In [ ]:
# train for bot 1000 stocks


end_of_train=pd.to_datetime('2006-01-31')
start_of_validation=end_of_train
end_of_validation=start_of_validation+pd.DateOffset(years=3)
start_of_test=end_of_validation
end_of_test=start_of_test+pd.DateOffset(years=1)



cycle_counter=0

cycle_r_2_results_bot_1000={}

cycle_prediction_results_bot_1000={}

while end_of_test<=pd.to_datetime('2020-12-31'):
  
  
  print(f'Cycle({cycle_counter}) starts')
#--------------------------------------------------- cycle data prep step ---------------------------------------------- 
  
  cycle_train_val=bot_1000_df.loc[bot_1000_df.index.get_level_values(1)<end_of_validation]
  
  cycle_test=bot_1000_df.loc[(bot_1000_df.index.get_level_values(1)>=start_of_test) & (bot_1000_df.index.get_level_values(1)<end_of_test)]
  
  

  cycle_train_val=cycle_train_val.assign(test_fold=-1)
  cycle_train_val.loc[cycle_train_val.index.get_level_values(1)>=start_of_validation,'test_fold']=0



  




#--------------------------------------------------- cycle model training step ---------------------------------------------- 

  cv = model_selection.PredefinedSplit(cycle_train_val['test_fold'])
  
  enet=linear_model.ElasticNetCV(cv=cv,random_state=344, max_iter=10000)
  enet.fit(X=cycle_train_val.drop(columns=['test_fold', 'retadj']),y=cycle_train_val['retadj'])

  print(f'Cycle{cycle_counter} model trained')

  

  
#--------------------------------------------------- cycle model prediction step ---------------------------------------------- 
  monthly_r_2=0
  count=0
  
  # iteratively predict every month in test set 
  for date,df in cycle_test.groupby('DateYM'):
   
     count+=1
     
     y_pred=enet.predict(X=df.drop(columns=['retadj']))
     
     monthly_r_2+=metrics.r2_score(y_true=df['retadj'], y_pred=y_pred)

     cycle_prediction_results_bot_1000[df.index.get_level_values(1)[0]]=y_pred
  
  
  cycle_r_2_results_bot_1000[f'Cycle{cycle_counter}Average Monthly R2:']=(monthly_r_2/count)

  

  print(f'Cycle{cycle_counter} prediction done')
  

  
#---------------------------------------------------  rolling dates updating step ---------------------------------------------- 
  
  cycle_counter+=1
  

  # move the end of TRAINING set 1 more year to include one more year from the start(2000). VALIDATION set start point and TEST set start point will move back one year subsequently
  end_of_train=end_of_train+pd.DateOffset(years=1)
  start_of_validation=end_of_train
  end_of_validation=start_of_validation+pd.DateOffset(years=3)
  start_of_test=end_of_validation
  end_of_test=start_of_test+pd.DateOffset(years=1)



Cycle(0) starts
Cycle0 model trained
Cycle0 prediction done
Cycle(1) starts
Cycle1 model trained
Cycle1 prediction done
Cycle(2) starts
Cycle2 model trained
Cycle2 prediction done
Cycle(3) starts
Cycle3 model trained
Cycle3 prediction done
Cycle(4) starts
Cycle4 model trained
Cycle4 prediction done
Cycle(5) starts
Cycle5 model trained
Cycle5 prediction done
Cycle(6) starts
Cycle6 model trained
Cycle6 prediction done
Cycle(7) starts
Cycle7 model trained
Cycle7 prediction done
Cycle(8) starts
Cycle8 model trained
Cycle8 prediction done
Cycle(9) starts
Cycle9 model trained
Cycle9 prediction done
Cycle(10) starts
Cycle10 model trained
Cycle10 prediction done


In [ ]:
cycle_prediction_results_bot_1000

{Timestamp('2009-01-31 00:00:00'): array([ 0.00755856,  0.00387117,  0.00598316,  0.00502338,  0.00707947,
         0.00531563,  0.00514795,  0.00676727,  0.00707947,  0.00653491,
         0.00502338,  0.00447802,  0.00102858,  0.00102858,  0.00707947,
         0.00598316,  0.00102858,  0.00447802,  0.00102858, -0.00143633,
         0.00767514,  0.00447802,  0.00598316,  0.00489722,  0.00554799,
         0.00665069,  0.00458981,  0.00102858,  0.00746674,  0.00554799,
         0.00707947,  0.00489722,  0.00531563,  0.00531563, -0.00143633,
         0.00429676,  0.00352543,  0.00755856,  0.00598316,  0.00102858,
         0.00352543,  0.00688864, -0.00164953,  0.00489722,  0.00352543,
         0.00447802,  0.00598316,  0.007331  ,  0.00102858,  0.0074356 ,
         0.00447802,  0.00344798,  0.00489722,  0.00565339,  0.00102858,
         0.00792187,  0.00102858,  0.00102858,  0.00767514,  0.00683594,
         0.00779012,  0.00665069,  0.00102858,  0.00531563,  0.00102858,
         0.005983

In [ ]:
cycle_r_2_results_bot_1000

{'Cycle0Average Monthly R2:': -0.12039685569752545,
 'Cycle1Average Monthly R2:': -0.1411906186748398,
 'Cycle2Average Monthly R2:': -0.18100065398115817,
 'Cycle3Average Monthly R2:': -0.06454651054709028,
 'Cycle4Average Monthly R2:': -0.02900704944120931,
 'Cycle5Average Monthly R2:': -0.05001078343268258,
 'Cycle6Average Monthly R2:': -0.0646790171111388,
 'Cycle7Average Monthly R2:': -0.05572692148645161,
 'Cycle8Average Monthly R2:': -0.026481507653848185,
 'Cycle9Average Monthly R2:': -0.10925790246694424,
 'Cycle10Average Monthly R2:': -0.03153980005587547}